### 载入pos和unlabeled数据

In [1]:
path_all = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/"
train_pos_features_f = path_all + "tpp_features/12*/part-*"
train_unlabled_features_f = path_all + "train_unlabled_features/part-*"
test_pos_features_f = path_all + "test_pp_features/1231/part-*"
test_unlabled_features_f = path_all + "test_unlabled_features/part-*"

In [2]:
train_pos_features = sc.textFile(train_pos_features_f).coalesce(50).map(lambda x: (eval(x)[0], [i for y in eval(x)[1] for i in y ]))
train_unlabled_features = sc.textFile(train_unlabled_features_f).coalesce(100).map(lambda x: (eval(x)[0], [i for y in eval(x)[1] for i in y ]))
test_pos_features = sc.textFile(test_pos_features_f).coalesce(50).map(lambda x: (eval(x)[0], [i for y in eval(x)[1] for i in y ]))
test_unlabled_features = sc.textFile(test_unlabled_features_f).coalesce(100).map(lambda x: (eval(x)[0], [i for y in eval(x)[1] for i in y ]))

In [4]:
"""
print(train_pos_features.count())  # 235586
print(train_unlabled_features.count())  # 8268837
print(test_pos_features.count())  # 77653
print(test_unlabled_features.count())  # 8342347
"""

235586
8268837
77653
8342347


### 从unlabled中抽取alfa进行处理

In [6]:
alfa = 0.1

In [7]:
# 从unlabled中抽取alfa作为negative
### train_neg_features = random.sample(train_unlabled_features.collect(), int(alfa * 8268837))  # 太慢了
# train_neg_features = train_unlabled_features.takeSample(False, int(alfa * 8268837))
# sc.parallelize(train_neg_features).coalesce(10).saveAsTextFile(path_all + "train_neg_features_01")
train_neg_features = sc.textFile(path_all + "train_neg_features_01").map(lambda x: eval(x))

In [8]:
# test_neg_features = random.sample(test_unlabled_features.collect(), int(alfa * 8342347))
# test_neg_features = test_unlabled_features.takeSample(False, int(alfa * 8342347))
# sc.parallelize(test_neg_features).coalesce(10).saveAsTextFile(path_all + "test_neg_features_01")
test_neg_features = sc.textFile(path_all + "test_neg_features_01").map(lambda x: eval(x))

### 从pos中取一部分作为spy加入unlabeled中构建PU数据集

In [9]:
blta = 0.8

In [10]:
pnsplit = int(train_pos_features.count()*blta)   # pnsplit以前的部分作为pu_pos, 以后的部分作为pu_neg

In [12]:
pu_pos_features = train_pos_features.collect()[:pnsplit]
pu_neg_features = train_pos_features.collect()[pnsplit:] + train_neg_features.collect()

In [13]:
print(len(pu_pos_features), len(pu_neg_features))

188468 874001


### 从PU数据集抽取gamma部分作为PU训练，其余作为PU测试

In [14]:
gamma=0.6

In [15]:
pu_train_pos = sc.parallelize(pu_pos_features, 50).takeSample(False, int(len(pu_pos_features) * gamma))  # list
# random.sample(pu_pos_features, int(len(pu_pos_features) * gamma))
# pu_test_pos = [item for item in pu_pos_features if item not in pu_train_pos]
pu_test_pos = sc.parallelize(pu_pos_features).subtractByKey(sc.parallelize(pu_train_pos)).collect()

In [16]:
print(len(pu_train_pos), len(pu_test_pos))

113080 75388


In [17]:
# pu_train_neg = random.sample(pu_neg_features, int(len(pu_neg_features) * gamma))
# pu_test_neg = [item for item in pu_neg_features if item not in pu_train_neg]
pu_train_neg = sc.parallelize(pu_neg_features, 50).takeSample(False, int(len(pu_neg_features) * gamma))  # list
pu_test_neg = sc.parallelize(pu_neg_features).subtractByKey(sc.parallelize(pu_train_neg)).collect()

In [18]:
print(len(pu_train_neg), len(pu_test_neg))

524400 349601


### PU训练

In [19]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier


In [21]:
pu_train_data = sc.parallelize(pu_train_pos).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
            .union(sc.parallelize(pu_train_neg).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

In [22]:
pu_test_data = sc.parallelize(pu_test_pos).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
            .union(sc.parallelize(pu_test_neg).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

#### LR

In [23]:
publr = LogisticRegression(maxIter=15, regParam=0.024).fit(pu_train_data)

In [24]:
pu_test_data_preRes = publr.transform(pu_test_data.select("features")).join(pu_test_data, 'features')

In [25]:
aaa = pu_test_data_preRes.filter(pu_test_data_preRes.prediction == 0).filter(pu_test_data_preRes.label == 0)\
.select('features', 'probability').collect()

In [26]:
predict_to_0_prob = []
for i in aaa:
    if i.features[-1] != 0 and i.features[-2] != 0:
        predict_to_0_prob.append(i.probability[0])

In [27]:
print(len(predict_to_0_prob), max(predict_to_0_prob), min(predict_to_0_prob), sum(predict_to_0_prob)/len(predict_to_0_prob))

5881 0.955686527802 0.500010760552 0.683705352903


In [28]:
bbb = pu_test_data_preRes.filter(pu_test_data_preRes.label == 0).select('features', 'probability').collect()

In [29]:
RN = []
for i in bbb:
    if i.probability[0] > max(predict_to_0_prob):
        RN.append(i.features)

In [59]:
print(len(RN), RN[4])

90595 [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,5.0,3.0,8.0,3.0,3.0,5.0,7.0,10.0,8.0,0.0,11.0,6.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,10.0,9.0,7.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]


In [31]:
train_data = train_pos_features.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
            .union(sc.parallelize(RN).map(lambda x: Row(label=0.0, features=x))).toDF()

In [34]:
test_data = test_pos_features.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
            .union(test_neg_features.map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()
test_data.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.0,0.0,0.0,0.0,...|  1.0|
|[1.0,0.0,0.0,0.0,...|  1.0|
|[0.0,0.0,0.0,0.0,...|  1.0|
+--------------------+-----+
only showing top 3 rows



#### 对第4天进行预测

In [35]:
LR =  LogisticRegression(maxIter=15, regParam=0.024).fit(train_data)

In [36]:
predict_4th_day = LR.transform(test_data.select('features')).join(test_data, 'features')

In [37]:
predict_4th_day.show(3)

+--------------------+--------------------+--------------------+----------+-----+
|            features|       rawPrediction|         probability|prediction|label|
+--------------------+--------------------+--------------------+----------+-----+
|[0.0,0.0,0.0,0.0,...|[-1.0493904964668...|[0.25934215940090...|       1.0|  1.0|
|[0.0,0.0,0.0,0.0,...|[0.30648903945096...|[0.57602804497085...|       0.0|  0.0|
|[0.0,0.0,0.0,0.0,...|[0.35946059400152...|[0.58890985280857...|       0.0|  0.0|
+--------------------+--------------------+--------------------+----------+-----+
only showing top 3 rows



In [38]:
accuracy = (predict_4th_day.filter(predict_4th_day.prediction == predict_4th_day.label).count())/(predict_4th_day.count())

In [39]:
accuracy

0.9577931304217632

#### without PU

In [ ]:
train_data_noPU = train_pos_features.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
            .union(sc.parallelize(train_neg_features.takeSample(False, int(250000)))\
                   .map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

In [ ]:
train_data_noPU.show(3)

In [ ]:
train_data_noPU.count()

In [ ]:
LR_noPU = LogisticRegression(maxIter=15, regParam=0.024).fit(train_data_noPU)

In [ ]:
predict_4th_day_noPU = LR_noPU.transform(test_data.select('features')).join(test_data, 'features')

In [ ]:
accuracy_noPU = (predict_4th_day_noPU.filter(predict_4th_day_noPU.prediction == predict_4th_day_noPU.label).count())\
/(predict_4th_day_noPU.count())

In [ ]:
accuracy_noPU